In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import spikeinterface as si
import matplotlib.pyplot as plt
import os
from matplotlib.backends.backend_pdf import PdfPages
import spikeinterface.extractors as se

In [3]:
trigger_time = pd.read_csv("/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/03_single_neuron/trigger_time_refine.csv", index_col=0)

In [5]:
import probeinterface as pi
from probeinterface.plotting import plot_probe_group
probe_30channel = pi.read_probeinterface("/media/ubuntu/sda/data/probe.json")

In [6]:
import spikeinterface.preprocessing as spre
date_order = ['032322', '042322', '052322', '052422', '062422', '072422', '082422', '092222', '102522',
              '122322', '012123', '022423', '032323', '042323', '052423', '062323', '072123']
record_dir = os.listdir("/media/ubuntu/sda/data/mouse11/ns4/natural_image")
for date in date_order:
    record = [s for s in record_dir if date in s][0]
    reocording = se.read_blackrock(f'/media/ubuntu/sda/data/mouse11/ns4/natural_image/{record}').remove_channels(["98", '31', '32'])
    reocording = reocording.set_probegroup(probe_30channel)
    reocording = spre.bandpass_filter(reocording, freq_min=300, freq_max=3000)
    reocording = spre.common_reference(reocording, reference="global", operator="median")
    trigger_temp = trigger_time[trigger_time['date'] == int(date)]
    for i in range(len(trigger_temp)):
        temp = reocording.get_traces(start_frame=trigger_temp.iloc[i, 0],
                                end_frame=trigger_temp.iloc[i, 1])
        np.save(f'/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/04_get_neuronal_channel/seg/seg_{date}_{trigger_temp.iloc[i, 4]}.npy', temp)
        